# Trabalho prático 3

O código de Huffman: implementar o algoritmo de Huffman e testar na compactação de sequência de caracteres.

## Huffman


O código de Huffman é um algoritmo de compressão de dados sem perdas. A ideia é associar a largura variada de códigos aos caracteres de entrada, a largura dos códigos associados é proporcional a frequência dos caracteres de entrada.

Os códigos associados são chamados de códigos de prefixo, tais são códigos binários os quais garantem que um código não seja prefixo de outro.

O tempo de complexidade do algoritmo de Huffman implementado abaixo é de O(nLogn).

1.  Crie duas filas de prioridades vazias.
2.  Crie um nó folha para cada caractere e coloque-o na primeira fila em ordem crescente de frequência. Inicialize a segunda fila como vazia.
3.  Retire os dois nós com as menores frequências examinando a frente de ambas as filas. Repetindo os seguintes passos duas vezes:

        a. Se a segunda fila está vazia, retire da primeira fila.
        b. Se a primeira fila está vazia, retire da segunda fila.
        c. Senão, compare a frente das duas filas e retire o nó mínimo.

4.  Crie um novo nó interno com a frequência igual a soma das frequêcias dos dois nós. Faça o primeiro nó retirado como o filho esquerdo e o segundo nó retirado como filho direito. Enfileire este nó na segunda fila.
5.  Repita os passos 3 e 4 enquanto houver pelo menos um nó nas filas. O nó restante é o nó raíz e completamos a árvore.


Para alcançarmos a implementaçao de Huffman criaremos uma classe para as filas de prioridades `Queue` e para os nós de Huffman `Node`:

In [255]:
# Classe para representar um nó da árvore de Huffman
class Node:

    def __init__(self, data=None, freq=None,
                 left=None, right=None):
        self.data = data
        self.freq = freq
        self.left = left
        self.right = right
        self.huff = ''

    # Função para checar se o nó é uma folha
    def isLeaf(self):
        return (self.left == None and
                self.right == None)

# Classe para representar uma fila de prioridade


class Queue:

    def __init__(self):
        self.queue = []

    # Função para checar se a fila tem tamanho 1
    def isSizeOne(self):
        return len(self.queue) == 1

    # Função para checar se a fila está vazia
    def isEmpty(self):
        return self.queue == []

    # Função para adicionar um item à fila
    def enqueue(self, x):
        self.queue.append(x)

    # Função para remover um item da fila
    def dequeue(self):
        return self.queue.pop(0)

    def printQueue(self):
        print("Fila: ", end="")
        for i in self.queue:
            print(i.freq, end=" ")

        print("") 

# Função para extrair o nó de menor frequência das filas


def findMin(firstQueue, secondQueue):

    # Passo 3.1: Se a primeira fila estiver vazia,
    # remova da segunda fila
    if secondQueue.isEmpty():
        return [firstQueue.dequeue(), "first"]

    # Passo 3.2: Se a segunda fila estiver vazia,
    # remova da primeira fila
    if firstQueue.isEmpty():
        return [secondQueue.dequeue(), "second"]

    # Passo 3.3: Se a frequência do primeiro item da
    # primeira fila for menor que a frequência do primeiro
    # item da segunda fila, remova da primeira fila
    if (firstQueue.queue[0].freq <
            secondQueue.queue[0].freq):
        return [firstQueue.dequeue(), "first"]

    # # Passo 3.4: Se a frequência do primeiro item da
    # # segunda fila for igual que a frequência do primeiro
    # # item da primeira fila, remova da primeira fila
    # if (firstQueue.queue[0].freq ==
    #         secondQueue.queue[0].freq):
    #     return [firstQueue.dequeue(), "first"]

    return [secondQueue.dequeue(), "second"]


# Função para imprimir os códigos de Huffman
def printCodes(root, arr):

    if root.left:
        arr.append(0)
        printCodes(root.left, arr)
        arr.pop(-1)

    if root.right:
        arr.append(1)
        printCodes(root.right, arr)
        arr.pop(-1)

    if root.isLeaf():
        print(f"'{root.data}': ", end="")
        print("'", end='')
        for i in arr:
            print(i, end="")

        print("',", end='')
        print()

# Função para imprimir a árvore de Huffman


def printTree(root: Node):
    def height(root: Node):
        return 1 + max(height(root.left), height(root.right)) if root else -1
    nlevels = height(root)
    width = pow(2, nlevels+1)

    q = [(root, 0, width, 'c')]
    levels = []

    while (q):
        node, level, x, align = q.pop(0)
        if node:
            if len(levels) <= level:
                levels.append([])

            levels[level].append([node, level, x, align])
            seg = width//(pow(2, level+1))
            q.append((node.left, level+1, x-seg, 'l'))
            q.append((node.right, level+1, x+seg, 'r'))

    for i, l in enumerate(levels):
        pre = 0
        preline = 0
        linestr = ''
        pstr = ''
        seg = width//(pow(2, i+1))
        for n in l:
            valstr = ''
            if n[0].data == "$":
                valstr = str(n[0].freq)
            else:
                valstr = str(n[0].data)
            if n[3] == 'r':
                linestr += ' '*(n[2]-preline-1-seg-seg//2) + \
                    '¯'*(seg + seg//2)+'\\'
                preline = n[2]
            if n[3] == 'l':
                linestr += ' '*(n[2]-preline-1)+'/' + '¯'*(seg+seg//2)
                preline = n[2] + seg + seg//2
            # correct the potition acording to the number size
            pstr += ' '*(n[2]-pre-len(valstr))+valstr
            pre = n[2]
        print(linestr)
        print(pstr)

Com isso, chegamos na função `Huffman` que monta a árvore de Huffman.

In [256]:
# Constroi a arvore de Huffman e retorna a raiz
# da arvore e imprime os codigos de Huffman de modo
# transversal
def Huffman(data, freq, size):

    # Passo 1: Crie uma fila vazia
    firstQueue = Queue()
    secondQueue = Queue()

    k = 0
    print("-------------------------")
    print(f"i = {k}")
    print("")

    # Ordenando os nós em ordem crescente de frequência
    nos = []
    for i in range(size):
        nos.append(Node(data[i], freq[i]))
    
    nos.sort(key=lambda x: x.freq, reverse=False)

    print("Nos ordenados: ")
    for i in range(size):
        print(f"{nos[i].data}", end='    ')

    print("")
    # Passo 2: Adicione cada nó na primeira fila
    for i in range(size):
        firstQueue.enqueue(nos[i])
        print(f"{nos[i].data}", end='    ')

    print("")

    # Roda até que haja apenas um nó na segunda fila e
    # todos os nós tenham sido removidos da primeira fila
    # pois o ultimo nó da segunda fila será a raiz da arvore
    while not (firstQueue.isEmpty() and
               secondQueue.isSizeOne()):
        
        firstQueue.printQueue()
        secondQueue.printQueue()

        k += 1
        # Passo 3: Retire os dois nós de menor
        # frequência da fila 1 e 2 e crie um
        # novo nó interno com a soma das duas
        # frequências. Enfileire este nó na fila 2.
        left, queue1 = findMin(firstQueue, secondQueue)
        right, queue2 = findMin(firstQueue, secondQueue)

        # if queue1 == queue2:
        #     if right.freq == left.freq:
        #         aux = right
        #         right = left
        #         left = aux

        left.huff = 0
        right.huff = 1

        # Passo 4: Crie um novo nó interno com
        # frequência igual à soma das frequências
        # dos dois nós de menor frequência
        # removidos.
        top = Node("$", left.freq + right.freq,
                   left, right)
        print("-------------------------")
        print(f"i = {k}")
        print(f"        {top.freq}       ")
        print(f"       /  \\       ")
        print(
            f"      {left.freq if left.data == '$' else left.data}   {right.freq if right.data == '$' else right.data}      ")
        secondQueue.enqueue(top)
        printTree(top)

    print("-------------------------")
    root = secondQueue.dequeue()

    # Impressão dos códigos e árvore de Huffman
    arr = []
    printTree(root)
    printCodes(root, arr)

    return root


### Codificação na Língua Portuguesa X Língua Inglesa

Sabendo a tabela de frequências das letras do alfabeto na língua portuguesa, podemos comparar o tamanho médio dos códigos gerados por Huffman com o tamanho tradicional (para 26 letras são necessários 5 bits), digamos que estamos com um espaço de 100 letras: 

In [257]:
tabela_freq_portugues = {
    'a': 0.1463,
    'b': 0.0104,
    'c': 0.0388,
    'd': 0.0499,
    'e': 0.1257,
    'f': 0.0102,
    'g': 0.0130,
    'h': 0.0128,
    'i': 0.0618,
    'j': 0.0040,
    'k': 0.0002,
    'l': 0.0278,
    'm': 0.0474,
    'n': 0.0505,
    'o': 0.1073,
    'p': 0.0252,
    'q': 0.0120,
    'r': 0.0653,
    's': 0.0781,
    't': 0.0434,
    'u': 0.0463,
    'v': 0.0167,
    'w': 0.0001,
    'x': 0.0021,
    'y': 0.0001,
    'z': 0.0047,
}

In [258]:
soma_freq = 0
for i in tabela_freq_portugues:
    soma_freq += tabela_freq_portugues[i]

print(f"Soma das frequências: {soma_freq}")

Soma das frequências: 1.0001


Analogamente, descrevemos a tabela de frequências para a língua inglesa:

In [259]:
tabela_freq_inglesa = {
    'a': 0.08167,
    'b': 0.01492,
    'c': 0.02782,
    'd': 0.04253,
    'e': 0.12702,
    'f': 0.02228,
    'g': 0.02015,
    'h': 0.06094,
    'i': 0.06966,
    'j': 0.00153,
    'k': 0.00772,
    'l': 0.04025,
    'm': 0.02406,
    'n': 0.06749,
    'o': 0.07507,
    'p': 0.01929,
    'q': 0.00095,
    'r': 0.05987,
    's': 0.06327,
    't': 0.09056,
    'u': 0.02758,
    'v': 0.00978,
    'w': 0.02360,
    'x': 0.00150,
    'y': 0.01974,
    'z': 0.00074,
}

**Aplicando a tabela da língua portuguesa em Huffman.**

In [260]:
arr = list(tabela_freq_portugues.keys())
freq = list(tabela_freq_portugues.values())

for i in range(len(freq)):
    freq[i] = freq[i] * 10000

size = len(arr)

Huffman(arr, freq, size)


-------------------------
i = 0

Nos ordenados: 
w    y    k    x    j    z    f    b    q    h    g    v    p    l    c    t    u    m    d    n    i    r    s    o    e    a    
w    y    k    x    j    z    f    b    q    h    g    v    p    l    c    t    u    m    d    n    i    r    s    o    e    a    
Fila: 1.0 1.0 2.0 21.0 40.0 47.0 102.00000000000001 104.0 120.0 128.0 130.0 167.0 252.0 278.0 388.0 434.0 463.0 474.0 499.0 505.00000000000006 618.0 653.0 781.0 1073.0 1257.0 1463.0000000000002 
Fila: 
-------------------------
i = 1
        2.0       
       /  \       
      w   y      

 2.0
 /¯ ¯\
 w   y
Fila: 2.0 21.0 40.0 47.0 102.00000000000001 104.0 120.0 128.0 130.0 167.0 252.0 278.0 388.0 434.0 463.0 474.0 499.0 505.00000000000006 618.0 653.0 781.0 1073.0 1257.0 1463.0000000000002 
Fila: 2.0 
-------------------------
i = 2
        4.0       
       /  \       
      2.0   k      

     4.0
   /¯¯¯ ¯¯¯\
 2.0       k
 /¯ ¯\
 w   y
Fila: 21.0 40.0 47.0 102.00000000000001 1

In [261]:
# m: 0000
# z: 0001000
# k: 0001001000
# y: 00010010010
# w: 00010010011
# x: 000100101
# j: 00010011
# q: 000101
# p: 00011
# d: 0010
# n: 0011
# o: 010
# h: 011000
# g: 011001
# l: 01101
# i: 0111
# e: 100
# r: 1010
# v: 101100
# f: 1011010
# b: 1011011
# c: 10111
# a: 110
# s: 1110
# t: 11110
# u: 11111

Assim, temos a tabela de códigos:

In [262]:
tabela_huffman_portugues = {
  'm': '0000',
  'z': '0001000',
  'k': '0001001000',
  'y': '00010010010',
  'w': '00010010011',
  'x': '000100101',
  'j': '00010011',
  'q': '000101',
  'p': '00011',
  'd': '0010',
  'n': '0011',
  'o': '010',
  'h': '011000',
  'g': '011001',
  'l': '01101',
  'i': '0111',
  'e': '100',
  'r': '1010',
  'v': '101100',
  'f': '1011010',
  'b': '1011011',
  'c': '10111',
  'a': '110',
  's': '1110',
  't': '11110',
  'u': '11111'
}

In [264]:
valores = list(tabela_huffman_portugues.values())
soma = 0
for valor in valores:
  soma += len(valor)
print(soma)
print(f"média: {soma/len(valores)}")

152
média: 5.846153846153846


**Aplicando a tabela da língua inglesa em Huffman.**

In [265]:
arr = list(tabela_freq_inglesa.keys())
freq = list(tabela_freq_inglesa.values())
size = len(arr)

Huffman(arr, freq, size)

-------------------------
i = 0

Nos ordenados: 
z    q    x    j    k    v    b    p    y    g    f    w    m    u    c    l    d    r    h    s    n    i    o    a    t    e    
z    q    x    j    k    v    b    p    y    g    f    w    m    u    c    l    d    r    h    s    n    i    o    a    t    e    
Fila: 0.00074 0.00095 0.0015 0.00153 0.00772 0.00978 0.01492 0.01929 0.01974 0.02015 0.02228 0.0236 0.02406 0.02758 0.02782 0.04025 0.04253 0.05987 0.06094 0.06327 0.06749 0.06966 0.07507 0.08167 0.09056 0.12702 
Fila: 
-------------------------
i = 1
        0.00169       
       /  \       
      z   q      

0.00169
 /¯ ¯\
 z   q
Fila: 0.0015 0.00153 0.00772 0.00978 0.01492 0.01929 0.01974 0.02015 0.02228 0.0236 0.02406 0.02758 0.02782 0.04025 0.04253 0.05987 0.06094 0.06327 0.06749 0.06966 0.07507 0.08167 0.09056 0.12702 
Fila: 0.00169 
-------------------------
i = 2
        0.0030299999999999997       
       /  \       
      x   j      

0.0030299999999999997
 /¯ ¯\
 x   j

In [266]:
tabela_huffman_inglesa = {
  't': '000',
  'v': '001000',
  'z': '001001000',
  'q': '001001001',
  'x': '001001010',
  'j': '001001011',
  'k': '0010011',
  'f': '00101',
  'w': '00110',
  'm': '00111',
  'u': '01000',
  'c': '01001',
  'r': '0101',
  'h': '0110',
  's': '0111',
  'e': '100',
  'n': '1010',
  'i': '1011',
  'b': '110000',
  'p': '110001',
  'y': '110010',
  'g': '110011',
  'o': '1101',
  'a': '1110',
  'l': '11110',
  'd': '11111'
}

In [267]:
valores = list(tabela_huffman_inglesa.values())

soma = 0
for valor in valores:
  soma += len(valor)

print(soma)
print(f"média: {soma/len(valores)}")

142
média: 5.461538461538462
